In [1]:
import tensorflow as tf

print(tf.__version__)

2.1.0


In [2]:
!pip install tensorflow==2.1.0
!pip install tensorflow-gpu==2.1.0

     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 3.9MB 43.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 450kB 40.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=9967f89413cd0b314352225d34da23717425ce020d412a7a1de5a81969664205
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

     |████████████████████████████████| 421.8MB 24kB/s 


In [2]:
!pip install pythainlp

     |████████████████████████████████| 10.6MB 4.0MB/s 
     |████████████████████████████████| 747kB 38.3MB/s 


In [3]:
# clone the project with dataset
!git clone https://github.com/wingedRuslan/ai-palette-test.git

Cloning into 'ai-palette-test'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 24 (delta 2), reused 17 (delta 1), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [4]:
# unzip dataset
!unzip ai-palette-test/train_thai_processed_dataset_approach_3.zip

Archive:  ai-palette-test/train_thai_processed_dataset_approach_3.zip
  inflating: train_thai_processed_dataset_approach_3.csv  


In [5]:
import pandas as pd
import numpy as np

from pythainlp import word_tokenize

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [6]:
# read training dataset
dataset_df = pd.read_csv('train_thai_processed_dataset_approach_3.csv')

In [7]:
dataset_df.shape

(39723, 3)

In [8]:
# shuffle
# shuffle rows in the dataset
dataset_df = dataset_df.sample(frac=1).reset_index(drop=True)

In [9]:
# bag-of-words
vectorizer = CountVectorizer(max_features=7000, tokenizer=word_tokenize, 
                             encoding='utf-8', max_df=0.9, min_df=1)

In [10]:
dataset_encoded = vectorizer.fit_transform(np.array(dataset_df['Text'])).todense()

In [11]:
dataset_encoded.shape

(39723, 7000)

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
target_encoder = OneHotEncoder()

In [14]:
target_encoded = target_encoder.fit_transform(np.array(dataset_df['Rating']).reshape(-1,1)).todense()

# np.array(dataset_df['Rating']).reshape(-1,1)

In [15]:
target_encoded.shape

(39723, 5)

In [16]:
# split train-dev-test
X_train, X_test, Y_train, Y_test = train_test_split(dataset_encoded, target_encoded, test_size=0.15, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [17]:
X_train.shape, Y_train.shape

((30387, 7000), (30387, 5))

In [18]:
X_val.shape, Y_val.shape

((3377, 7000), (3377, 5))

In [19]:
X_test.shape, Y_test.shape

((5959, 7000), (5959, 5))

## Modeling

In [20]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 128-dimensional vector
x = layers.Embedding(input_dim=7000, output_dim=128)(inputs)

# Add 1 bidirectional LSTMs
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

# Add a classifier
outputs = layers.Dense(5, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         896000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 995,461
Trainable params: 995,461
Non-trainable params: 0
_________________________________________________________________


### Train and evaluate the model

In [21]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [22]:
X_train = tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(X_train))
X_val = tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(X_val))

In [21]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_val, Y_val))

Train on 30387 samples, validate on 3377 samples
Epoch 1/5
30387/30387 [==============================] - 1363s 45ms/sample - loss: 1.2167 - accuracy: 0.4664 - val_loss: 1.1879 - val_accuracy: 0.4735
Epoch 2/5
30387/30387 [==============================] - 1356s 45ms/sample - loss: 1.2091 - accuracy: 0.4689 - val_loss: 1.1859 - val_accuracy: 0.4735
Epoch 3/5
30387/30387 [==============================] - 1360s 45ms/sample - loss: 1.2089 - accuracy: 0.4689 - val_loss: 1.1915 - val_accuracy: 0.4726
Epoch 4/5
30387/30387 [==============================] - 1365s 45ms/sample - loss: 1.2078 - accuracy: 0.4691 - val_loss: 1.1854 - val_accuracy: 0.4729
Epoch 5/5
30387/30387 [==============================] - 1363s 45ms/sample - loss: 1.2074 - accuracy: 0.4696 - val_loss: 1.1867 - val_accuracy: 0.4729


## Test dataset Predictions

In [22]:
# read test dataset
test_df = pd.read_csv('ai-palette-test/test.csv', sep=';')

In [23]:
# convert to vectors
test_encoded = vectorizer.transform(np.array(test_df['review'])).todense()

In [26]:
# make prediction for each data sample in test dataset
test_predicitons = []

for test_review in test_encoded:
  test_predicitons.append(model.predict(test_review))

In [30]:
# choose the most probable class
final_prediction = []

for prediction in test_predicitons:
  max_index = prediction.argmax()

  final_prediction.append(max_index+1)

In [32]:
# prepare submission file
submission_BiLSTM_df = pd.DataFrame({'reviewID':test_df.reviewID,
                          'rating':final_prediction})
submission_BiLSTM_df.to_csv('submission_BiLSTM.csv',index=False)

In [33]:
# !cp -r bert_task_output/ drive/MyDrive/ai-palette-test/

cp: cannot stat 'bert_task_output/': No such file or directory
